<a href="https://colab.research.google.com/github/HerbertDaca/projekt_IL2/blob/main/Sie%C4%87_Neuronowa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tweepy
import pandas as pd
import matplotlib.pyplot as plt
import regex as re
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding, Dense, Dropout, Input, LSTM, GlobalMaxPool1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import Constant
import spacy
import wandb
wandb.login(key= "bdd577b3c2ee944f33f7fc061b8def672936904f")
from wandb.keras import WandbCallback


In [ ]:

academic_bearer = "AAAAAAAAAAAAAAAAAAAAADIEawEAAAAAxzzD4cQ2g8FGK2%2BkKz2%2FJvTnoMA%3D09uegYs5HrQvrsFkAEl3WwxhspBYFBIH3Vnykec79asqiUsSoA"
client = tweepy.Client(academic_bearer)



In [ ]:




RETWEETED_REGEX = r"^RT @([^ ]+):"
USER_REGEX = r"@([^ ]+)"
URL_REGEX = r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)"

def clean_tweet(tweet):
    is_retweet = False if re.match(RETWEETED_REGEX, tweet) is None else True
    di = {"is_retweet": is_retweet,
          "retweeted_user": re.findall(RETWEETED_REGEX, tweet)[0] if is_retweet else "",
          "links": re.findall(URL_REGEX, tweet)}
    tweet = re.sub(RETWEETED_REGEX, "", tweet)
    di["mentioned_users"] = re.findall(USER_REGEX, tweet)
    tweet = re.sub(USER_REGEX, "", tweet)
    tweet = re.sub(URL_REGEX, "", tweet)
    tweet = re.sub("\n", " ", tweet)
    di["cleaned_tweet"] = tweet.strip()
    return di



In [ ]:


tweet_fields = ['created_at', 'public_metrics']
query = 'place_country:PL lang:PL'



max_result = 500
number_of_wanted_tweets = 1000
licznik = 0

file_name2 = "many_tweets2.tsv"
file_name = "many_tweets.tsv"
with open(file_name, 'w+', encoding= "utf-8") as filehandle:
    with open(file_name2, 'w+', encoding="utf-8") as filehandle2:
        for tweet in tweepy.Paginator(client.search_all_tweets, query=query, tweet_fields=tweet_fields, max_results=max_result, sort_order = "relevancy").flatten(limit=number_of_wanted_tweets):
            tweet_dict = clean_tweet(tweet.text)
            tweet_czysty = tweet_dict["cleaned_tweet"]
            licznik += 1
            filehandle.write(f"{tweet_czysty}\t{tweet.id}\n")
            filehandle2.write(
                f"{6 * tweet.public_metrics['reply_count'] + tweet.public_metrics['like_count'] + 6 * tweet.public_metrics['retweet_count']} ,"
                f"{tweet.id} ,"
                f"{tweet.created_at} ,"
                f"{licznik}\n")



In [ ]:

df = pd.read_csv(filepath_or_buffer="C:/Users/PC/PycharmProjects/TWEEPY/many_tweets2.tsv", names= ("points", "tweet_id", "created_at", "nr"))



print(df)

In [ ]:
kwantyl = df.points.quantile(0.95)
df2=df.assign(labelki=lambda x: x.points > kwantyl)
df2['labelki'] = df2['labelki'].astype(int)

df2.drop(df.iloc[:, 0:4], inplace=True, axis=1)

df2.to_csv(path_or_buf="C:/Users/PC/PycharmProjects/TWEEPY/many_tweets3.tsv", encoding="utf-8", header= False, index_label= False, index= False)

In [ ]:


train_labels = pd.read_csv("C:/Users/PC/PycharmProjects/TWEEPY/many_tweets3.tsv")
train_data = pd.read_csv("C:/Users/PC/PycharmProjects/TWEEPY/many_tweets.tsv", sep = "\t").dropna()

print(train_data.iloc[:, 0])

# nlp = en_core_web_lg.load()
# Vectorizer = TextVectorization()
# x = Vectorizer.adapt(test_data.iloc[:, 0].to_numpy())
# vocab = Vectorizer.get_vocabulary()
# print(x)




In [ ]:

Vectorizer = TextVectorization()

Vectorizer.adapt(train_data.iloc[:,0].to_numpy())
vocab = Vectorizer.get_vocabulary()
print(len(vocab))
num_tokens = len(vocab)

for i, word in enumerate(vocab):
    embedding_matrix[i] = nlp(str(word)).vector